In [2]:
import numpy as np

In [8]:
class Loss:
    def __init__(self, lossType = "MSE"):
        self.lossType = lossType 
    
    def backwardPass(self, X, Y):
        if(self.lossType == "MSE"):
            return self.backwardPassMSE(X, Y)
        elif(self.lossType == "CE"):
            return self.backwardPassCE(X,Y)
        return
    
    def forwardPass(self, X, Y):
        if(self.lossType == "MSE"):
            return self.forwardPassMSE(X, Y)
        elif(self.lossType == "CE"):
            return self.forwardPassCE(X,Y)
        return
   
    def forwardPassMSE(X, Y):
        return ((X-Y).T)@(X-Y)*0.5
    
    def forwardPassCE(X, Y):
        return -np.sum(Y*np.log(X))
   
    def backwardPassMSE(P, Y):
        return P-Y
    
    def backwardPassCE(P,Y):
        # dl_dx = -dl_dn * (Y/X);
        return Y/P
    


In [ ]:
class Activation:
    def __init__(self, act = "linear"):
        self.act = act 
    
    def backwardPass(self, X):
        if(self.lossType == "linear"):
            return self.backwardPassLinear(X)
        elif(self.lossType == "sigmoid"):
            return self.backwardPassSigmoid(X)
        return
    
    def forwardPass(self, X):
        if(self.lossType == "linear"):
            return self.forwardPassLinear(X)
        elif(self.lossType == "sigmoid"):
            return self.forwardPassSigmoid(X)
        return
   
    def forwardPassLinear(X):
        return X
    
    def forwardPassSigmoid(X):
        return 1/(1 + np.exp(-X))
   
    def backwardPassLinear(P):
        return 1
    
    def backwardPassSigmoid(self, P):
        return self.forwardPassSigmoid(P) * (1 - self.forwardPassSigmoid(P))
    



In [ ]:
class Layer:
    def __init__(self, numNeurons = 1, numNeuronsPrev = 1, act = "linear", alpha = 1e-5):
        # Neurons in the current layer
        self.numNeurons = numNeurons
        # Neurons in the previous layer
        self.numNeuronsPrev = numNeuronsPrev
        # Weights for linear sum
        self.W = np.random.rand(numNeurons, numNeuronsPrev)
        # Bias of size Nx1
        self.B = np.random.rand(numNeurons, 1)
        # Activation function 
        self.activation = Activation(act)
        self.alpha = alpha
        self.input = None
        self.output = None

    def update(self, dl_dw, dl_db):
        self.W -= self.alpha*(dl_dw)
        self.B -= self.alpha*(dl_db)
    def forwardPasslinear(self, X):
        self.input = X
        return (self.W)@X + (self.B)
    
    def forwardPass(self, X):
        arg = self.forwardPasslinear(X)
        return self.activation.forwardPass(arg)
    
    def backwardPasslinear(self, dl_ds):
        # dl_dx = np.zeros((dl_dw.shape[1], 1))
        dl_dx = self.W.T@dl_ds
        dl_dw = (dl_ds@(self.input).T)
        return (dl_dx, dl_dw)
    
    def backwardPass(self, dl_dh):
        # dl_ds where S = W@X + B
        dl_ds = dl_dh*self.activation.backwardPass(self.forwardPasslinear(self.input))  
        dl_dx, dl_dw = self.backwardPasslinear(dl_ds)

        self.update(dl_dw, dl_ds)
        return dl_dx

        # fw= self.forwardPasslinear(self.input)
        # dl_dx = dl_dh * (self.activation.backwardPass(fw))
        # return self.backwardPasslinear(dl_dx)

In [ ]:
class NeuralNetwork:
    def __init__(self, listLayers = [], str = "MSE", inp = np.zeros((10,5)), pred = np.ones((10,1))):
        self.listLayers = listLayers
        self.lossObj = Loss(str)
        self.target = pred
        # Input data to the neural network
        self.input = inp
    
    def forwardPassNN(self, i, Y):
        R = self.listLayers[i].forwardPass(Y)
        return R

    def run(self):
        for d in range(len(self.inp)):
            Y = self.inp[d].reshape(self.inp.shape[1], 1)
            for i in range(len(self.listLayers)):
                Y = self.forwardPassNN(i, Y)
            
            targ = self.target[d].reshape(self.target.shape[1],1)
            myLoss = self.lossObj.forwardPass(Y, self.target[d])
            dl_dh = self.lossObj.backwardPass(Y, self.target[d])
            for i in range(len(self.listLayers) -1, -1, -1):
                dl_dh = self.listLayers[i].backwardPass(dl_dh)
    
    def test(self, dataPts, targets):
        accuracy = 0 
        for ind in range(len(dataPts)):
            Y = dataPts[ind]
            for i in range(len(self.listLayers)):
                Y = self.forwardPassNN(i)
            
            if Y.item() == targets[ind]:
                accuracy += 1
        print(100*(accuracy/len(dataPts)))   


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
Y = raw_df.values[1::2, 2]
Y = Y.reshape((Y.shape[0],1))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)